In [14]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


In [15]:
#Khởi tạo mediapie
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Hàm tính khoảng cách

In [16]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    '''
    Calculate angle between 3 points in 3D space using dot product.
    '''
    A = np.array(pointA)
    B = np.array(pointB)
    C = np.array(pointC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle


## 3. Lỗi gối quá hẹp 
Vì tỉ lệ giữa gối và hông khi lên, xuống trong 1 chu kỳ là khác nhau, ta cần xác định giai đoạn của độngtacs trước khi xét


In [17]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [18]:
#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()

In [19]:
#import model để dự đoán giai đoạn
import pickle


with open("LR_Up_Down_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

Xem data chuẩn

In [20]:
#Làm sạch dữ liệu dataframe trước khi thêm dữ liệu mới
df = pd.DataFrame(columns=["video", "shoulder_width", "feet_width"])

In [21]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def process_frame(Video_folder, Video_name):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính khoảng cách
            knee_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách đầu gối, chân
                landmarks = results.pose_landmarks.landmark

                # Lấy trạng thái hiện tại 
                # predicted_class = trained_model.predict(X)[0]
                # prediction_probability = trained_model.predict_proba(X)[0]

                # current_state ="middle"
                # if prediction_probability[prediction_probability.argmax()] >= 0.7: # Nếu xác suất dự đoán lớn hơn 0.7 thì mới cập nhật trạng thái
                #     if predicted_class == 0:
                #         current_state = "down"
                #     elif current_state == "middle" and predicted_class == 1:
                #         current_state = "up"
                # else:
                #     current_state = "middle"

                # Khoảng cách giữa 2 đầu gối 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                knee_width = calculate_distance(left_knee, right_knee)

                # Khoảng cách giữa 2 chân
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)


                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee distance
                cv2.putText(image, "KNEE", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display current stage
                cv2.putText(image, "STAGE", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                # cv2.putText(image, current_state, (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            df.loc[len(df)] = [Video_name, knee_width, feet_width]

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# process_frame(video_folder, video_files)



In [22]:
import os

# Đường dẫn thư mục chứa video
video_folder = "Data/Train/Correct"
# video_folder = "Data/Train/Dau_goi_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

df = pd.DataFrame(columns=["video", "knee_width", "feet_width"])

for video in video_files:
    process_frame(video_folder, video)

KeyboardInterrupt: 

In [ ]:
df 

,video,state,knee_width,feet_width
0,20250228_080823000_iOS (video-converter.com).mp4,middle,0.100038,0.072577
1,20250228_080823000_iOS (video-converter.com).mp4,middle,0.101382,0.074630
2,20250228_080823000_iOS (video-converter.com).mp4,middle,0.103591,0.075260
3,20250228_080823000_iOS (video-converter.com).mp4,middle,0.106241,0.075738
4,20250228_080823000_iOS (video-converter.com).mp4,middle,0.111362,0.075840
...,...,...,...,...
6660,VID_20250307_091538.mp4,up,0.078873,0.089440
6661,VID_20250307_091538.mp4,up,0.079007,0.089385
6662,VID_20250307_091538.mp4,up,0.078910,0.089124
6663,VID_20250307_091538.mp4,up,0.079104,0.089858


In [ ]:
#Tính toán tỉ lệ giữa vai và chân
df["ratio"] = df["knee_width"] / df["feet_width"]
df

,video,state,knee_width,feet_width,ratio
0,20250228_080823000_iOS (video-converter.com).mp4,middle,0.100038,0.072577,1.378368
1,20250228_080823000_iOS (video-converter.com).mp4,middle,0.101382,0.074630,1.358459
2,20250228_080823000_iOS (video-converter.com).mp4,middle,0.103591,0.075260,1.376443
3,20250228_080823000_iOS (video-converter.com).mp4,middle,0.106241,0.075738,1.402740
4,20250228_080823000_iOS (video-converter.com).mp4,middle,0.111362,0.075840,1.468383
...,...,...,...,...,...
6660,VID_20250307_091538.mp4,up,0.078873,0.089440,0.881857
6661,VID_20250307_091538.mp4,up,0.079007,0.089385,0.883901
6662,VID_20250307_091538.mp4,up,0.078910,0.089124,0.885389
6663,VID_20250307_091538.mp4,up,0.079104,0.089858,0.880324


In [ ]:
df.describe()
df

In [13]:
#Phân tích bộ data
df_group = df.groupby("state").describe()
# Lọc chỉ các cột có mean, min, max
df_filtered = df_group.loc[:, (slice(None), ['count', 'mean', 'min', 'max'])]

# Hiển thị kết quả
print(df_filtered)

       knee_width                               feet_width            \
            count      mean       min       max      count      mean   
state                                                                  
down        633.0  0.135702  0.099111  0.171286      633.0  0.078301   
middle     4241.0  0.122063  0.065207  0.170036     4241.0  0.086818   
up         1791.0  0.074527  0.056432  0.115880     1791.0  0.077080   

                             ratio                                
             min       max   count      mean       min       max  
state                                                             
down    0.063325  0.099697   633.0  1.743373  1.313757  2.277239  
middle  0.060602  0.106113  4241.0  1.415321  0.810242  2.132808  
up      0.055971  0.099924  1791.0  0.975980  0.749651  1.447330  


# Kết luận:
Với ratio = knee/ feet <br>
Khi stage = up: ratio < 0.74: gối hẹp <br>
Khi stage = middle: ratio < 0.81: gối hẹp <br>
Khi stage = down: ratio < 1.31: gối hẹp <br>
